In [277]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.python.framework import ops
import tensorflow.keras as keras

np.random.seed(1)

In [ ]:
# Function for loading dataset
def load_dataset():
    
    # load training set
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_x_raw = np.array(train_dataset["train_set_x"][:])
    train_y_raw = np.array(train_dataset["train_set_y"][:])
    
    # load test set
    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_x_raw = np.array(test_dataset["test_set_x"][:])
    test_y_raw = np.array(test_dataset["test_set_y"][:])
    
    # get list of classes (0 to 5)
    classes = np.array(test_dataset["list_classes"][:])
    
    # reshape y sets into [1, # of samples]
    train_y_raw = train_y_raw.reshape((1, train_y_raw.shape[0]))
    test_y_raw = test_y_raw.reshape((1, test_y_raw.shape[0]))
    
    return train_x_raw, train_y_raw, test_x_raw, test_y_raw, classes

In [ ]:
# Load data
X_train_raw, Y_train_raw, X_test_raw, Y_test_raw, classes = load_dataset()

In [ ]:
# Function for one-hot-encoding Y datasets
def one_hot_encode(Y, num_classes):
    Y = np.eye(num_classes)[Y.reshape(-1)].T
    return Y

In [ ]:
# Normalize X datasets (255 RGB values)
X_train = X_train_raw / 255
X_test = X_test_raw / 255

# One-Hot Encode Y datasets
Y_train = one_hot_encode(Y_train_raw, 6).T
Y_test = one_hot_encode(Y_test_raw, 6).T

In [ ]:
# Function for initializing placeholders X, Y 
def init_placeholders(n_h_raw, n_w_raw, n_c_raw, n_y_raw):
    
    X = tf.placeholder(tf.float32, shape=(None, n_h_raw, n_w_raw, n_c_raw), name="X")
    Y = tf.placeholder(tf.float32, shape=(None, n_y_raw), name="Y")
    
    return X, Y

In [ ]:
# Function for initializing parameters W1, W2 using xavier/GlorotUniform initialization
def init_parameters(w1_hw, w1_c, w1_n, w2_hw, w2_c, w2_n):
    
    tf.set_random_seed(1)
    
    W1 = tf.get_variable("W1", [w1_hw, w1_hw, w1_c, w1_n], initializer = keras.initializers.GlorotUniform(seed=0))
    W2 = tf.get_variable("W2", [w2_hw, w2_hw, w2_c, w2_n], initializer = keras.initializers.GlorotUniform(seed=0))
    
    parameters = {"W1" : W1, 
                  "W2" : W2}
    
    return parameters

In [ ]:
# Function for forward propagation
def fwd_prop(X, parameters):
    
    # Get parameters
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    ### LAYER I ###
    # 2D Convolutional
    Z1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding="SAME")
    # ReLU
    A1 = tf.nn.relu(Z1)
    # Max Pooling
    P1 = tf.nn.max_pool(A1, ksize=[1,8,8,1], strides=[1,8,8,1], padding="SAME")
    
    ### LAYER II ###
    # 2D Convolutional
    Z2 = tf.nn.conv2d(P1, W2, strides=[1,1,1,1], padding="SAME")
    # ReLU
    A2 = tf.nn.relu(Z2)
    # Max Pooling
    P2 = tf.nn.max_pool(A2, ksize=[1,4,4,1], strides=[1,4,4,1], padding="SAME")
    
    ### LAYER III ###
    # Flatten
    F = keras.layers.Flatten()(P2)
    # Fully-Connected
    Z3 = keras.layers.Dense(6, activation=None)(F)

    return Z3

In [ ]:
# Compute the cost
def compute_cost(Z3, Y):
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=Y))
    
    return cost